In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from keras import Sequential,losses
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.optimizers import SGD,Adam
from keras.utils import np_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")

In [ ]:
print("Count of Images Before deduplicating process",len(df))
df.drop_duplicates(inplace=True)
print("Count of Images After deduplicating process",len(df))

In [ ]:
datagen = ImageDataGenerator(width_shift_range=4,height_shift_range=4,validation_split=0.2)
X = df.loc[:,"0.1":]
y = df.loc[:,"0"]
X=X.values.reshape(len(X),28,28,1)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.90)
X_train = X_train/255
X_test = X_test/255
y_train=np_utils.to_categorical(y_train)

In [ ]:
model=Sequential() # A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
model.add(Conv2D(8,(3,3),activation="relu")) #2D convolution layer
model.add(MaxPooling2D(strides=1))
model.add(Conv2D(16,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=1))
model.add(Conv2D(24,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=1))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(26,activation="softmax"))
model.compile(loss=losses.categorical_crossentropy,metrics=["accuracy"],optimizer=SGD(learning_rate=0.07))

In [ ]:
checkpoint = ModelCheckpoint(filepath="model_sgd.pkl",monitor="val_accuracy",verbose=2,save_best_only=True,mode="max")
model.fit(datagen.flow(X_train, y_train, batch_size=256,subset="training"),
          validation_data=datagen.flow(X_train, y_train, batch_size=256,subset="validation"),
          epochs=50,batch_size=256,verbose=2,callbacks=[checkpoint])

In [ ]:
model.summary()

In [ ]:
y_predict=model.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

In [ ]:
print(classification_report(y_predict,y_test))

In [ ]:
import shutil
shutil.make_archive("model_sgd.pkl", 'zip', "/kaggle/working/model_sgd.pkl")

In [ ]:
from keras.models import load_model
model1=load_model("/kaggle/working/model_sgd.pkl")

In [ ]:
y_predict=model1.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

In [ ]:
print(classification_report(y_predict,y_test))